# **klue/roberta-large**

In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 데이터 로드

In [3]:
train_df = pd.read_csv('/content/drive/MyDrive/GBT 해커톤/data/train_df_1007.csv')
test_df = pd.read_csv('/content/drive/MyDrive/GBT 해커톤/data/test_df_1007.csv')

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54314 entries, 0 to 54313
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      54314 non-null  object
 1   분류      54314 non-null  object
 2   제목      54314 non-null  object
 3   키워드     54314 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB


In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   제목      23405 non-null  object
 2   키워드     23405 non-null  object
dtypes: object(3)
memory usage: 548.7+ KB


### 모델링

- epoch: 10
- learning rate: 2e-5
- batch size: 16
- max length: 256

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
from types import SimpleNamespace
from torch.utils.data import DataLoader, WeightedRandomSampler

In [7]:
config = {
    "learning_rate": 2e-5,
    "epoch": 10,
    "batch_size": 16
}

CFG = SimpleNamespace(**config)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')
model = AutoModelForSequenceClassification.from_pretrained('klue/roberta-large', num_labels=len(train_df['분류'].unique())).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item] if self.labels is not None else -1
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [10]:
# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

# 데이터 분할 (train -> train + validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

# 데이터셋 생성
train_dataset = TextDataset(train_df.키워드.tolist(), train_df.label.tolist(), tokenizer)
val_dataset = TextDataset(val_df.키워드.tolist(), val_df.label.tolist(), tokenizer)
test_dataset = TextDataset(test_df.키워드.tolist(), None, tokenizer)  # 라벨 없음

# 각 클래스의 샘플 수를 계산
class_counts = train_df['label'].value_counts().sort_index()
class_weights = 1.0 / class_counts
sample_weights = train_df['label'].map(class_weights)

# WeightedRandomSampler 생성
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# 데이터 로더 생성 (WeightedRandomSampler 사용)
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, sampler=sampler)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

In [11]:
train_dataset[3]

{'text': '찐윤 수식어 이원모 산단 반도체 사활 총선 국민 용인갑 후보 이원모 대통령실 비서관 중앙 지방정부 소통 반도체 산단 적임자 경쟁국 반도체 파격투자 세액공제 대통령 반도체 산업 중요성 강조 국가산업단지 진짜 반도체 국가 산업 단지 사활 이원모 대통령실 인사 비서관 국민 후보 용인시갑 추천 전략공천 윤석열 대통령 조언 비서관 본인 강남을 공천 신청 용산 출신 인사들 양지행 비판 결정 수용 의지 공천 공관위 결정 용인갑 산업 우리나라 미래 명운 면적 차지 처인구 반도체 클러스터 국가산업단지 삼성전자 SK하이닉스 투자 이원모 대통령실 인사 비서관 처인구 선거사무소 이데일리 인터뷰 이원모 캠프 비서관 업무 연고 강조 개인 연고 대통령실 근무 정부 부처 산하 기관 대통령 인사 보좌 소통 업무 전담 산업 단지 완공 국가 현안 뒷받침 의미 공관위 결정 중간 통보 용인갑 느낌 국가산단 프로젝트 정부 지방자치단체 조율 관계자들 연락 소통 세계 반도체 산업 규모 예산 우리나라 기준 사이 산단 조성 장기 프로젝트 초기 추진력 중요 바퀴 부연 비서관 현안 해결 찐윤 핵심 윤석열계 수식어 활용 사단 검찰 재직 윤석열 막내 이명박 대통령 비자금 사건 박근혜 대통령 국정농단 대통령 국정 농단 조국 입시비리 법무 장관 자녀 입시 비리 문재인 의혹 대통령 시절 월성 원자력 발전소 경제 조작 수사 검사 사직 법률지원팀 대선 캠프 법률 대통령 복심 인사비서관 발탁 찐윤 현실 정치 고민 그간 행적 총선 출마 자연 탈원전 수사 탈원전 조국 장관 수사 적폐 낙인 원칙 원자력발전소 재정비 정권 정지 그땐 애국심 취임 윤석열 대통령 여소야대 정부 여당 입법 발목 자연 총선 도전 비서관 장관 조국혁신당 지지율 정당 지지율 지지율 비례 지지율 약진 국민들 현명 판단 비서관 국회 입성 법안 조세특례제한법 원안 반도체특별법 K칩스법 조세 특레 제한 원안 세액공제 기본 세액 공제 중견기업 대기업 중견 기업 중소기업 세액 공제 야당 반대 대기업 중견기업 중견 기업 중소기업 통과 반도체 산업 세계 

In [12]:
# 옵티마이저 및 학습 파라미터 설정
optimizer = AdamW(model.parameters(), lr=CFG.learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
# 초기화
patience = 2  # 개선되지 않을 경우 기다리는 에폭 수
best_loss = np.inf  # 최상의 손실값 초기화
best_f1 = 0.0  # 최상의 F1 스코어 초기화
counter = 0  # 카운터 초기화

for epoch in range(CFG.epoch):
    model.train()
    epoch_loss = 0.0  # 에폭 손실 초기화

    # 학습 단계
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epoch}'):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()  # 에폭 손실 누적

    # Validation
    model.eval()
    val_loss = 0.0
    val_predictions = []
    val_true_labels = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            _, preds = torch.max(outputs.logits, dim=1)
            val_predictions.extend(preds.cpu().tolist())
            val_true_labels.extend(labels.cpu().tolist())

    # 평균 검증 손실 계산
    val_loss /= len(val_loader)

    # F1 스코어 계산
    current_f1 = f1_score(val_true_labels, val_predictions, average='macro')

    # Early stopping 체크
    if val_loss < best_loss:
        best_loss = val_loss  # 최상의 손실값 갱신
        counter = 0  # 카운터 초기화
    else:
        counter += 1  # 카운터 증가

    # F1 스코어가 개선되면 모델 저장
    if current_f1 > best_f1:
        best_f1 = current_f1  # 최상의 F1 스코어 갱신
        torch.save(model.state_dict(), f'model_best_f1.pth')  # 모델 저장
        print(f"Model saved with F1 Score: {best_f1:.4f}")

    print(f"Epoch {epoch + 1}, Validation Loss: {val_loss:.4f}, F1 Score: {current_f1:.4f}")

    # Early stopping이 활성화되면 훈련 종료
    if counter >= patience:
        print("Early stopping triggered. Training stopped.")
        break

Validating: 100%|██████████| 679/679 [08:10<00:00,  1.38it/s]


Model saved with F1 Score: 0.5467
Epoch 1, Validation Loss: 1.5872, F1 Score: 0.5467


Validating: 100%|██████████| 679/679 [08:10<00:00,  1.38it/s]


Model saved with F1 Score: 0.5741
Epoch 2, Validation Loss: 1.6676, F1 Score: 0.5741


Validating: 100%|██████████| 679/679 [08:11<00:00,  1.38it/s]

Epoch 3, Validation Loss: 1.6188, F1 Score: 0.5714
Early stopping triggered. Training stopped.


In [14]:
# 테스트 세트 추론
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().tolist())

# 라벨 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]

Testing: 100%|██████████| 1463/1463 [17:38<00:00,  1.38it/s]


In [15]:
sample_submission = pd.read_csv("/content/drive/MyDrive/GBT 해커톤/data/sample_submission.csv")
sample_submission["분류"] = decoded_predictions

sample_submission.to_csv("/content/drive/MyDrive/GBT 해커톤/data/submission_KLUE-RoBERTa-large_sampler_1008.csv", encoding='UTF-8-sig', index=False)

In [16]:
result = pd.read_csv("/content/drive/MyDrive/GBT 해커톤/data/submission_KLUE-RoBERTa-large_sampler_1008.csv")
result.head()

,ID,분류
0,TEST_00000,사회:사회일반
1,TEST_00001,사회:사회일반
2,TEST_00002,정치:행정_자치
3,TEST_00003,경제:취업_창업
4,TEST_00004,경제:경제일반


In [17]:
result['분류'].value_counts()

,count
분류,
지역,4572
사회:사회일반,1909
경제:부동산,1785
경제:반도체,1227
경제:경제일반,1086
사회:사건_사고,1036
사회:교육_시험,976
사회:의료_건강,712
정치:국회_정당,637
